In [5]:
import os
import pprint
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad


from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap



importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(runrecord_clirad)
importlib.reload(utils_clirad)
importlib.reload(setup_overlap)
importlib.reload(setup_bestfit)
importlib.reload(pipeline_lblnew)
importlib.reload(bestfit)
importlib.reload(runrecord_bestfit)
importlib.reload(runrecord_overlap)

<module 'climatools.lblnew.runrecord_overlap' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/runrecord_overlap.py'>

In [6]:
def params_clirad_nonoverlap():
    '''
    Define clirad-lw input parameter dictionaries
    for 2xn2o, n2o-only runs.
    '''
    commitnumber = '523cbb7'
    molecule = {'n2o': 2 * 3.2e-7}
    bands = [3, 9] # these are the only bands with n2o
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    params = []
    for band in bands:
        for atmpro, tsfc in atmpros:
            param = {'commitnumber': commitnumber,
                     'molecule': molecule,
                     'band': [band],
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            params.append(param)
    return params



def param_lblnewbestfit():
    '''    
    lblnew-bestfit input parameter dictionaries
    matching those from clirad-lw's 2xn2o, 
    n2o-only runs.
    '''
    commitnumber = 'a22ab94'
    
    params_clirad = params_clirad_nonoverlap()
    record_bestfit = runrecord_bestfit.params()
    
    params = []
    for param_clirad in params_clirad:
        
        molecule = list(param_clirad['molecule'].keys())[0]
        conc = param_clirad['molecule']['n2o']
        band = param_clirad['band'][0]
        band = utils_clirad.mapband_new2old()[band]
        atmpro = param_clirad['atmpro']
        tsfc = param_clirad['tsfc']
        
        param_bestfit = [r for r in record_bestfit
                         if r['commitnumber'] == commitnumber
                         if r['molecule'] == molecule
                         if r['conc'] == conc
                         if r['band'] == band
                         if r['atmpro'] == atmpro
                         if r['tsfc'] == tsfc]
        
        assert len(param_bestfit) == 1
        param_bestfit = param_bestfit[0]
        params.append(param_bestfit)
        
    return params


def parampairs_nonoverlap():
    params_clirad = params_clirad_nonoverlap()
    params_lblnew = param_lblnewbestfit()
    
    assert len(params_clirad) == len(params_lblnew)
    
    return list(zip(params_clirad, params_lblnew))




In [7]:
def script_clirad_lblnew():
    '''
    Script to run clirad and then plot results against the 
    corresponding lblnew case.  This needs both input parameter
    dictionaries from clirad and lblnew.
    '''
    ps = parampairs_nonoverlap()
    
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps, 
                                                 setup=setup)
    
    for param, _ in ps:
        print(pipeline.nbviewer_url(param=param, setup=setup))
        print()


In [10]:
pprint.pprint(list(zip(*parampairs_nonoverlap()))[0])

({'atmpro': 'mls',
  'band': [3],
  'commitnumber': '523cbb7',
  'molecule': {'n2o': 6.4e-07},
  'tsfc': 294},
 {'atmpro': 'saw',
  'band': [3],
  'commitnumber': '523cbb7',
  'molecule': {'n2o': 6.4e-07},
  'tsfc': 257},
 {'atmpro': 'trp',
  'band': [3],
  'commitnumber': '523cbb7',
  'molecule': {'n2o': 6.4e-07},
  'tsfc': 300},
 {'atmpro': 'mls',
  'band': [9],
  'commitnumber': '523cbb7',
  'molecule': {'n2o': 6.4e-07},
  'tsfc': 294},
 {'atmpro': 'saw',
  'band': [9],
  'commitnumber': '523cbb7',
  'molecule': {'n2o': 6.4e-07},
  'tsfc': 257},
 {'atmpro': 'trp',
  'band': [9],
  'commitnumber': '523cbb7',
  'molecule': {'n2o': 6.4e-07},
  'tsfc': 300})


In [9]:
script_clirad_lblnew()

Submitting radiation calculation for cases
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [3],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [3],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [9],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [9],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [9],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 300}

Submitting analysis for cases
{'atmpro': 'mls',
 'band': [3],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [3],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-07},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [3],
 'commitnumber': '523cbb7',
 'molecule': {'n2o': 6.4e-